In [34]:
import torch
from transformers import pipeline
%load_ext rich

The rich extension is already loaded. To reload it, use:
  %reload_ext rich


In [12]:
classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.



[
    {'label': 'POSITIVE', 'score': 0.9598051905632019},
    {'label': 'NEGATIVE', 'score': 0.9994558691978455}
]

### Tokenizer

In [13]:
from transformers import AutoTokenizer

In [14]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer


DistilBertTokenizerFast(name_or_path='distilbert-base-uncased-finetuned-sst-2-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
        0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
        100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
        101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
        102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
        103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [21]:
raw_inputs = [
    "This is the first sentence.",
    "What does the second sentence say?",
    "Need a very long sentence to see how it is tokenized?",
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
inputs


{
    'input_ids': tensor([[  101,  2023,  2003,  1996,  2034,  6251,  1012,   102,     0,     0,
             0,     0,     0,     0,     0],
        [  101,  2054,  2515,  1996,  2117,  6251,  2360,  1029,   102,     0,
             0,     0,     0,     0,     0],
        [  101,  2342,  1037,  2200,  2146,  6251,  2000,  2156,  2129,  2009,
          2003, 19204,  3550,  1029,   102]]),
    'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
}

### Model

In [16]:
from transformers import AutoModel

In [19]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)
model


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [27]:
model(**inputs).last_hidden_state.shape

torch.Size([3, 15, 768])

### Classification

In [28]:
from transformers import AutoModelForSequenceClassification

In [29]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [44]:
raw_inputs = [
    "I am very happy to be learning Hugging Face today.",
    "This is so sad!",
    "Reports of the earthquake devastated the town yesterday.",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
output = model(**inputs)

In [45]:
output.logits


tensor([[-4.2581,  4.6127],
        [ 4.2791, -3.4862],
        [ 1.5164, -1.3071]], grad_fn=<AddmmBackward0>)

In [46]:
preds = torch.nn.functional.softmax(output.logits, dim=-1)
preds


tensor([[1.4042e-04, 9.9986e-01],
        [9.9958e-01, 4.2401e-04],
        [9.4393e-01, 5.6067e-02]], grad_fn=<SoftmaxBackward0>)

In [47]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [49]:
pipeline("sentiment-analysis")(raw_inputs)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.



[
    {'label': 'POSITIVE', 'score': 0.9998595714569092},
    {'label': 'NEGATIVE', 'score': 0.9995760321617126},
    {'label': 'NEGATIVE', 'score': 0.9439323544502258}
]